## Pre-requisites
1. Ollama installed
2. Model `llama3.2:3b-instruct-fp16` pulled

In [ ]:
import dspy

lm = dspy.OllamaLocal("gemma2:9b")
dspy.configure(lm=lm)

In [ ]:
lm("Hello world!")

## Dataset [google-research-datasets/go_emotions](https://research.google/blog/goemotions-a-dataset-for-fine-grained-emotion-classification/) from Hugging Face

In [ ]:
from dspy import datasets

dl = datasets.DataLoader()
dataset_emotions = dl.from_huggingface(
    "google-research-datasets/go_emotions", "simplified", split=["train", "test"]
)

In [ ]:
len(dataset_emotions["train"]), len(dataset_emotions["test"])

In [ ]:
dataset_emotions["test"][:3]

In [ ]:
emotions = open("../data/emotions.txt").read().splitlines() + ["neutral"]
idx2emotion = {i: t for i, t in enumerate(emotions)}
idx2emotion

In [ ]:
for d in dataset_emotions:
    for e in dataset_emotions[d]:
        for k in e:
            if k == "labels":
                e[k] = [idx2emotion[i] for i in e[k]]

In [ ]:
dataset_emotions["test"][:3]

In [ ]:
dataset_emotions["train"][10:13]

## Define a Signature for Emotion Classification

In [ ]:
class EmotionSignature(dspy.Signature):
    """
    Given a list of emotions, predict the most likely emotions of the input sentence.
    Emotions:
    =========
    admiration
    amusement
    anger
    annoyance
    approval
    caring
    confusion
    curiosity
    desire
    disappointment
    disapproval
    disgust
    embarrassment
    excitement
    fear
    gratitude
    grief
    joy
    love
    nervousness
    optimism
    pride
    realization
    relief
    remorse
    sadness
    surprise
    neutral
    Give the most likely emotions of the input sentence as comma separated list.
    Do not return any additional information.
    """

    sentence = dspy.InputField(desc="""Text to predict emotions""")
    emotion_labels = dspy.OutputField(desc="""Predicted emotions""")

In [ ]:
predictor = dspy.Predict(EmotionSignature)

In [ ]:
predictor(sentence=dataset_emotions["test"][3]["text"])

In [ ]:
dataset_emotions["test"][3]["labels"]

In [ ]:
class Emotion(dspy.Signature):
    """Classify emotion among sadness, joy, love, anger, fear, surprise."""

    sentence = dspy.InputField()
    sentiment = dspy.OutputField()


sentence = "i started feeling a little vulnerable when the giant spotlight started blinding me"  # from dair-ai/emotion

classify = dspy.Predict(Emotion)
classify(sentence=sentence)